# MAS Lab session

In [ ]:
pip install pygame

#### test if files can be loaded

In [2]:
import pygame
import os
def test_image_loading():
    """
    Test function to check if the wolf.png and sheep.png images can be loaded
    """
    pygame.init()  # Initialize pygame
    
    # List of image paths to test
    image_paths = [
        'sheep.png',
        'wolf.png',
        'plant.png'
    ]
    
    print("Testing image loading...")
    for path in image_paths:
        try:
            print(f"Trying to load: {path}")
            print(f"File exists: {os.path.exists(path)}")
            if os.path.exists(path):
                img = pygame.image.load(path)
                print(f"✓ Successfully loaded {path}")
                print(f"  Image size: {img.get_size()}")
            else:
                print(f"✗ File not found: {path}")
        except Exception as e:
            print(f"✗ Error loading {path}: {e}")
    
    print("\nCurrent working directory:", os.getcwd())
    print("\nListing files in current directory:")
    for file in os.listdir('.'):
        print(f"  {file}")
    
    print("\nListing files in parent directory:")
    try:
        for file in os.listdir('..'):
            print(f"  {file}")
    except Exception as e:
        print(f"Could not list parent directory: {e}")

# Run the test before starting the simulation
test_image_loading()

Testing image loading...
Trying to load: sheep.png
File exists: False
✗ File not found: sheep.png
Trying to load: wolf.png
File exists: False
✗ File not found: wolf.png
Trying to load: plant.png
File exists: False
✗ File not found: plant.png

Current working directory: /

Listing files in current directory:
  home
  usr
  .resolve
  bin
  sbin
  .file
  etc
  var
  Library
  System
  .VolumeIcon.icns
  private
  .vol
  Users
  Applications
  opt
  dev
  Volumes
  .nofollow
  tmp
  cores

Listing files in parent directory:
  home
  usr
  .resolve
  bin
  sbin
  .file
  etc
  var
  Library
  System
  .VolumeIcon.icns
  private
  .vol
  Users
  Applications
  opt
  dev
  Volumes
  .nofollow
  tmp
  cores


#### simulate

In [ ]:
import pygame
import random
import math
import os

# Define constants for the screen width and height
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 1000

class Agent(pygame.sprite.Sprite):
    def __init__(self, size, color, x=None, y=None, image_path=None):
        super().__init__()

        # Load image if provided, otherwise create a circle
        if image_path and os.path.exists(image_path):
            # Load image and scale it
            original_image = pygame.image.load(image_path).convert_alpha()
            # Scale the image to size*2 x size*2 pixels
            self.surf = pygame.transform.scale(original_image, (size*4, size*4))
            self.original_image = self.surf  # Store for rotation
        else:
            # Create default circle
            self.surf = pygame.Surface((2*size, 2*size), pygame.SRCALPHA, 32)
            pygame.draw.circle(self.surf, color, (size, size), size)
            self.original_image = self.surf
            
        self.rect = self.surf.get_rect()

        # default values
        self.vmax = 2.0
        self.size = size

        # initial position
        self.x = x if x else random.randint(0, SCREEN_WIDTH)
        self.y = y if y else random.randint(0, SCREEN_HEIGHT)

        # initial velocity
        self.dx = 0
        self.dy = 0

        # initial values
        self.is_alive = True
        self.target = None
        self.age = 0
        self.energy = 0

        # move agent on screen
        self.rect.centerx = int(self.x)
        self.rect.centery = int(self.y)

    def update(self, screen, food=()):
        self.age = self.age + 1

        # update position based on delta x/y
        self.x = self.x + self.dx
        self.y = self.y + self.dy

        # ensure it stays within the screen window
        self.x = max(self.x, 0)
        self.x = min(self.x, SCREEN_WIDTH)
        self.y = max(self.y, 0)
        self.y = min(self.y, SCREEN_HEIGHT)

        # we can't move, just update the screen
        if self.vmax == 0:
            self.rect.centerx = int(self.x)
            self.rect.centery = int(self.y)
            screen.blit(self.surf, self.rect)
            return

        # initalize forces to zero
        fx = 0
        fy = 0
        
        # move in the direction of the target, if any
        if self.target:
            fx += 0.1*(self.target.x - self.x)
            fy += 0.1*(self.target.y - self.y)

        # update our direction based on the 'force'
        self.dx = self.dx + 0.05*fx
        self.dy = self.dy + 0.05*fy

        # slow down agent if it moves faster than it max velocity
        velocity = math.sqrt(self.dx ** 2 + self.dy ** 2)
        if velocity > self.vmax:
            self.dx = (self.dx / velocity) * (self.vmax)
            self.dy = (self.dy / velocity) * (self.vmax)

        # Rotate image to face movement direction if moving
        if velocity > 0.1:  # Only rotate if actually moving
            angle = math.degrees(math.atan2(-self.dy, self.dx))
            self.surf = pygame.transform.rotate(self.original_image, angle - 90)
            self.rect = self.surf.get_rect()

        # update graphics
        self.rect.centerx = int(self.x)
        self.rect.centery = int(self.y)
        screen.blit(self.surf, self.rect)

        # target is dead, don't chase it further
        if self.target and not self.target.is_alive:
            self.target = None

        # eat the target if close enough
        if self.target:
            squared_dist = (self.x-self.target.x)**2 + (self.y-self.target.y)**2
            if squared_dist < 400:
                self.target.is_alive = False
                self.energy = self.energy + 1

        # agent doesn't have a target, find a new one
        if not self.target:
            min_dist = 9999999
            min_agent = None
            for a in food:
                if a is not self and a.is_alive:
                    sq_dist = (self.x - a.x) ** 2 + (self.y - a.y) ** 2
                    if sq_dist < min_dist:
                        min_dist = sq_dist
                        min_agent = a

            if min_dist < 100000:
                self.target = min_agent


class Predator(Agent):
    def __init__(self, x=None, y=None):
        size = 10
        color = (255, 0, 0)
        # Use wolf.png image instead of a circle
        super().__init__(size, color, x, y, image_path="wolf.png")
        self.vmax = 2.5

class Prey(Agent):
    def __init__(self, x=None, y=None):
        size = 8
        color = (255, 255, 255)
        # Use sheep.png image instead of a circle
        super().__init__(size, color, x, y, image_path="sheep.png")
        self.vmax = 2.0

class Plant(Agent):
    def __init__(self, x=None, y=None):
        size = 10
        color = (0, 128, 0)
        super().__init__(size, color, x, y, image_path="plant.png")
        self.vmax = 0


def Main():
    # Import and initialize the pygame library
    pygame.init()
    clock = pygame.time.Clock()

    # Create the screen object
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption(f'Ecosystem Simulation')

    # create initial agents
    preys = [Prey() for i in range(10)]
    predators = [Predator() for i in range(10)]
    plants = [Plant() for i in range(100)]

    # Run until the user asks to quit
    while True:
        # Process inputs
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                raise SystemExit

        # Fill the background with a green-ish color (for grass)
        screen.fill((50, 120, 50))

        # update all agents
        [f.update(screen) for f in plants]
        [a.update(screen, food=plants) for a in preys]
        [a.update(screen, food=preys) for a in predators]

        # handle eaten and create new plant
        plants = [p for p in plants if p.is_alive is True]
        plants = plants + [Plant() for i in range(2)]

        # handle eaten and create new preys
        preys = [p for p in preys if p.is_alive is True]

        for p in preys[:]:
            if p.energy > 5:
                p.energy = 0
                preys.append(Prey(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20)))

        # handle old and create new predators
        predators = [p for p in predators if p.age < 2000]

        for p in predators[:]:
            if p.energy > 10:
                p.energy = 0
                predators.append(Predator(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20)))

        # Add display of counts
        font = pygame.font.SysFont(None, 24)
        stats = f"Wolves: {len(predators)}  Sheep: {len(preys)}  Plants: {len(plants)}"
        text = font.render(stats, True, (255, 255, 255))
        screen.blit(text, (10, 10))

        # draw all changes to the screen
        pygame.display.flip()
        clock.tick(24)         # wait until next frame (at 24 FPS)

    # Done! Time to quit.
    pygame.quit()


# Start the simulation
if __name__ == "__main__":
    Main()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-02-27 13:35:25.790 Python[59283:2980655] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-27 13:35:25.790 Python[59283:2980655] +[IMKInputSession subclass]: chose IMKInputSession_Modern


SystemExit: 

/Users/kaispeidel/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 